# Домашнее задание к теме Базовые понятия статистики

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики.

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

In [195]:
import pandas as pd
import numpy as np

In [200]:
# Список словарей с информацией для обработки данных.
# Ключи:
# - id (int): идентификатор
# - label (str): подпись
# - label_ru (str): подпись на русском
# - basis (bool): True — базовый атрибут, False — не базовый
# - type (string): тип атрибута. numerical — число, categorical — категориальный
# - values (dict): словарь сопоставления значений

attr = [
     {
     'id': 1,
     'label': 'surgery?',
     'label_ru': 'Операция',
     'basis': True,
     'type': 'categorical',
     'values': {
         '1': "Да, была операция",
         '2': "Нет, лечили без операции"
     }
     },
    {
     'id': 2,
     'label': 'Age',
     'label_ru': 'Возраст',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'Взрослая лошадь',
          '2': 'Молодая (< 6 месяцев)'
          }
     },
    {
     'id': 3,
     'label': 'Hospital Number',
     'label_ru': 'Номер госпитализации',
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 4,
     'label': 'rectal temperature',
     'label_ru': 'Ректальная температура',
     'basis': True,
     'type': 'numerical'
     },
    {
     'id': 5,
     'label': 'pulse',
     'label_ru': 'Пульс',
     'basis': True,
     'type': 'numerical'
     },
    {
     'id': 6,
     'label': 'respiratory rate',
     'label_ru': 'Частота дыхания',
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 7,
     'label': 'temperature of extremities',
     'label_ru': 'Температура конечностей',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'Нормальные',
          '2': 'Тёплые',
          '3': 'Прохладные',
          '4': 'Холодные'
          }
     },
    {
     'id': 8,
     'label': 'peripheral pulse',
     'label_ru': 'Периферический пульс',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'нормальный',
          '2': 'повышенный',
          '3': 'пониженный',
          '4': 'отсутствует'
          }
     },
    {
     'id': 9,
     'label': 'mucous membranes',
     'label_ru': 'Слизистые оболочки',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'нормально-розовые',
          '2': 'ярко-розовые',
          '3': 'бледно-розовые',
          '4': 'бледно-цианотичные',
          '5': 'ярко-красные / воспалённые',
          '6': 'тёмно-цианотичные'
          }
     },
    {
     'id': 10,
     'label': 'capillary refill time',
     'label_ru': 'Время капиллярного наполнения',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': '< 3 секунд',
          '2': '≥ 3 секунд'
          }
     },
    {
     'id': 11,
     'label': 'pain',
     'label_ru': 'Боль',
     'basis': True,
     'type': 'categorical',
     'values': {
          '1': 'бодрая, без боли',
          '2': 'подавленная',
          '3': 'перемежающаяся лёгкая боль',
          '4': 'перемежающаяся сильная боль',
          '5': 'постоянная сильная боль'
          }
     },
    {
     'id': 12,
     'label': 'peristalsis',
     'label_ru': 'Перистальтика',
     'basis': True,
     'type': 'categorical',
     'values': {
         '1': 'гипермоторика', 
          '2': 'нормальная', 
          '3': 'гипомоторика',
          '4': 'отсутствует'
          }
     },
    {
     'id': 13,
     'label': 'abdominal distension',
     'label_ru': 'Вздутие живота',
     'basis': True,
     'type': 'categorical',
     'values': {
          '1': 'нет',
          '2': 'лёгкое',
          '3': 'умеренное',
          '4': 'сильное'
          }
     },
    {
     'id': 14,
     'label': 'nasogastric tube',
     'label_ru': 'Назогастральная трубка',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'нет',
          '2': 'немного',
          '3': 'значительное количество'
          }
     },
    {
     'id': 15,
     'label': 'nasogastric reflux',
     'label_ru': 'Назогастральный рефлюкс',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'нет',
          '2': '> 1 литра',
          '3': '< 1 литра'
          }
     },
    {
     'id': 16,
     'label': 'nasogastric reflux pH',
     'label_ru': 'pH назогастрального рефлюкса',
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 17,
     'label': 'rectal examination - feces',
     'label_ru': 'Ректальный осмотр (кал)',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'нормальный',
          '2': 'увеличенный',
          '3': 'уменьшенный',
          '4': 'отсутствует'
          }
     },
    {
     'id': 18,
     'label': 'abdomen',
     'label_ru': 'Живот',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'нормальный',
          '2': 'другое', 
          '3': 'плотный кал в толстом кишечнике',  
          '4': 'вздутие тонкого кишечника',
          '5': 'вздутие толстого кишечника'
          }
     },
    {
     'id': 19,
     'label': 'packed cell volume',
     'label_ru': 'Гематокрит',
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 20,
     'label': 'total protein',
     'label_ru': 'Общий белок',
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 21,
     'label': 'abdominocentesis appearance',
     'label_ru': 'Внешний вид жидкости при абдоминоцентезе',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'прозрачная',
          '2': 'мутная',
          '3': 'серозно-кровянистая' 
          }
     },
    {
     'id': 22,
     'label': 'abdominocentesis total protein',
     'label_ru': 'Белок при абдоминоцентезе',
     'basis': False,
     'type': 'numerical'
     },
    {
     'id': 23,
     'label': 'outcome',
     'label_ru': 'Исход',
     'basis': True,
     'type': 'categorical',
     'values': {
          '1': 'выжила',
          '2': 'умерла',
          '3': 'эвтаназирована'
          }
     },
    {
     'id': 24,
     'label': 'surgical lesion?',
     'label_ru': 'Хирургическое поражение?',
     'basis': True,
     'type': 'categorical',
     'values': {
          '1': 'Да',
          '2': 'Нет'
     }
     },
    {
     'id': 25,
     'label': 'type of lesion',
     'label_ru': 'Тип поражения',

     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 26,
     'label': 'type of lesion',
     'label_ru': 'Тип поражения',
     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 27,
     'label': 'type of lesion',
     'label_ru': 'Тип поражения',

     'basis': False,
     'type': 'categorical'
     },
    {
     'id': 28,
     'label': 'cp_data',
     'label_ru': 'Наличие патологоанатомических данных',
     'basis': False,
     'type': 'categorical',
     'values': {
          '1': 'Да',
          '2': 'Нет'  
          }
    }
]

In [201]:
def prepare_data(file, attr, basis = False):
     """ Подготовить данные — из файла с учетом данных атрибутов и базиса атрибутов:
     - Оставить только требуемые атрибуты
     - Числовые атрибуты преобразовать в числа, остальные (категориальные) — в строки

     Аргументы:
     - file (str): имя файла
     - attr (list): список словарей
     - basis (bool): True — подготовить файл только с базовыми атрибутами, False (по-умолчанию) — со всеми атрибутами

     Возвращает:
     pandas.DataFrame: датафрейм с данными
     """
     df = pd.read_csv(file, header=None)
     data = pd.DataFrame()

     for column in df:   
          if (basis == True and attr[column]['basis'] == True) or (basis == False):
               if attr[column]['type'] == 'numerical':
                    data[column] = pd.to_numeric(df[column], errors='coerce')
               else:
                    data[column] = df[column].astype(str)

     return data

In [202]:
def replace_values(df, attr):
    """ Заменить значения — из словаря в данных атрибутов в датафрейме
    
    Аргументы:
    - df (pandas.DataFrame): датафрейм
    - attr (list): список словарей

    Возвращает:
    pandas.DataFrame: датафрейм с данными
    """
    for column in df:
        if 'values' in attr[column]:
            df[column] = df[column].map(attr[column]['values'])
    return df

In [203]:
def describe_data(df):
     """ Статистическое резюме датафрейма

     Аргументы:
     - df (pandas.DataFrame): датафрейм

     Возвращает:
     pandas.DataFrame: датафрейм с данными
     """

     data = pd.DataFrame()
     for column in df:
          described_column = pd.DataFrame(df[column].describe())
          data = pd.concat([data,described_column], axis=1)
     return data

In [204]:
def add_outliers(df):
    """ Добавить выбросы — в статистическое резюме

    Аргументы:
    - df (pandas.DataFrame): датафрейм

    Возвращает:
    pandas.DataFrame: датафрейм с данными

    """
    df.loc['iqr'] = None
    df.loc['lower'] = None
    df.loc['upper'] = None
    for column in df.columns:
        q1 = df[column]['25%']
        q3 = df[column]['75%']
        iqr = q3 - q1
        df.loc['iqr', column] = q3 - q1
        df.loc['lower', column] = q1 - 1.5 * iqr
        df.loc['upper', column] = q3 + 1.5 * iqr
    return df

In [205]:
def clear_missing(df, attr, basis = True, source = None):
    """ Работа с пропусками — заменяет NaN в ячейках датафрейма с учетом данных атрибутов, базиса атрибутов и столбца для основы оценки моды числовых данных:
    - Удалить ячейки с пустыми значениями, если в датафрейме только базовые атрибуты или не указан столбец для основы оценки моды числовых данных
    - В числовых ячейках заменить NaN на моду по группе
    - В остальных заменить NaN на 'Нет данных'

    Аргументы:
     - file (str): имя файла
     - attr (list): список словарей
     - basis (bool): True — подготовить датафрейм только с базовыми атрибутами, False (по-умолчанию) — со всеми атрибутами
     - source (str, int): столбец для основы оценки моды числовых данных:

    Возвращает:
    pandas.DataFrame: датафрейм с данными
    """
    if basis == True or source == None:
        data = df.dropna()
    else:
        data = df.dropna(subset=[source])
        outcome = df.groupby(source).agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        for column in data:
            if attr[column]['type'] == 'numerical':
                mask = data[column].isna()
                data.loc[mask, column] = data.loc[mask, source].map(lambda g: outcome.loc[g, column])
            else:
                mask = data[column].isna()
                data.loc[mask, column] = 'Нет данных'
    return data

In [206]:
def rename_columns(df, attr):
    """ Переименовать колонки — из информации об атрибутах

    Аргументы:
    - df (pandas.DataFrame): датафрейм
    - attr (list): список словарей

    Возвращает:
    pandas.DataFrame: датафрейм с данными
    """
    for column in df:
        df.rename(columns={column: attr[column]['label_ru']}, inplace=True)
    return df

In [207]:
# Задание 1
horses = prepare_data('horse_data.csv', attr, False)
horses = replace_values(horses, attr)
horses = describe_data(horses)
horses = rename_columns(horses, attr)
display(horses)

,Операция,Возраст,Номер госпитализации,Ректальная температура,Пульс,Частота дыхания,Температура конечностей,Периферический пульс,Слизистые оболочки,Время капиллярного наполнения,...,Гематокрит,Общий белок,Внешний вид жидкости при абдоминоцентезе,Белок при абдоминоцентезе,Исход,Хирургическое поражение?,Тип поражения,Тип поражения,Тип поражения,Наличие патологоанатомических данных
count,299,276,300,240.000000,276.000000,242.000000,244,231,253,266,...,271.000000,267.000000,135,102.000000,299,300,300,300,300,300
unique,2,1,284,NaN,NaN,NaN,4,4,6,2,...,NaN,NaN,3,NaN,3,2,61,6,2,2
top,"Да, была операция",Взрослая лошадь,532349,NaN,NaN,NaN,Прохладные,нормальный,нормально-розовые,< 3 секунд,...,NaN,NaN,мутная,NaN,выжила,Да,0,0,0,Нет
freq,180,276,2,NaN,NaN,NaN,109,115,79,188,...,NaN,NaN,48,NaN,178,191,56,293,299,201
mean,NaN,NaN,NaN,38.167917,71.913043,30.417355,NaN,NaN,NaN,NaN,...,46.295203,24.456929,NaN,3.019608,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,0.732289,28.630557,17.642231,NaN,NaN,NaN,NaN,...,10.419335,27.475009,NaN,1.968567,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,35.400000,30.000000,8.000000,NaN,NaN,NaN,NaN,...,23.000000,3.300000,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,37.800000,48.000000,18.500000,NaN,NaN,NaN,NaN,...,38.000000,6.500000,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,38.200000,64.000000,24.500000,NaN,NaN,NaN,NaN,...,45.000000,7.500000,NaN,2.250000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,38.500000,88.000000,36.000000,NaN,NaN,NaN,NaN,...,52.000000,57.000000,NaN,3.900000,NaN,NaN,NaN,NaN,NaN,NaN


# Задание 1

1. Операция — большинство случаев колик потребовали хирургического вмешательства, что указывает на тяжесть состояния.  
2. Возраст — выборка состоит почти исключительно из взрослых животных.  
3. Номер госпитализации — почти все записи относятся к разным случаям, дубликаты минимальны.  
4. Ректальная температура — в пределах нормы у большинства, повышена при инфекции, снижена при шоке.  
5. Пульс — выраженная тахикардия, отражает боль и шок.  
6. Частота дыхания — повышена, указывает на боль и компенсаторную гипервентиляцию.  
7. Температура конечностей — у большинства снижена, признак плохой перфузии и шока.  
8. Периферический пульс — чаще нормальный, но у части ослаблен при гипоперфузии.  
9. Слизистые оболочки — в норме у большинства, у части — признаки гипоксии и шока.  
10. Время капиллярного наполнения — в норме у большинства, удлинено при шоке.  
11. Боль — присутствует почти всегда, выраженность различна, сильная боль чаще требует операции.  
12. Перистальтика — снижена у большинства, что указывает на непроходимость.  
13. Вздутие живота — встречается часто, сильное вздутие требует хирургического вмешательства.  
14. Назогастральная трубка — объём жидкости > 1 л указывает на обструкцию кишечника.  
15. Количество жидкости при зондировании — большие объёмы связаны с непроходимостью.  
16. Ректальное исследование — часто выявляет вздутие и отсутствие кала при обструкции.  
17. Внешний вид живота — вздутие и напряжение характерны для острых случаев.  
18. Ректально обнаруженный кал — отсутствие кала — частый признак непроходимости.  
19. Гематокрит — повышен при обезвоживании и шоке.  
20. Общий белок — тенденция к повышению при дегидратации и воспалении.  
21. Внешний вид жидкости при абдоминоцентезе — мутная жидкость указывает на воспаление или некроз.  
22. Белок при абдоминоцентезе — высокий уровень связан с перитонитом и тяжёлыми поражениями.  
23. Исход — большинство лошадей выжили при своевременном лечении.  
24. Хирургическое поражение? — большинство случаев — хирургические патологии кишечника.  
25. Тип поражения (1) — разнообразные причины, чаще механические обструкции.  
26. Тип поражения (2) — уточняет локализацию или механизм поражения.  
27. Тип поражения (3) — вероятно, различает механические и немеханические формы.  
28. Наличие патологоанатомических данных — в трети случаев есть подтверждение диагноза после гибели.

In [208]:
# Задание 2
horses = prepare_data('horse_data.csv', attr, False)
horses = replace_values(horses, attr)
horses = describe_data(horses)
horses = add_outliers(horses)
horses = rename_columns(horses, attr)
display(horses)

,Операция,Возраст,Номер госпитализации,Ректальная температура,Пульс,Частота дыхания,Температура конечностей,Периферический пульс,Слизистые оболочки,Время капиллярного наполнения,...,Гематокрит,Общий белок,Внешний вид жидкости при абдоминоцентезе,Белок при абдоминоцентезе,Исход,Хирургическое поражение?,Тип поражения,Тип поражения,Тип поражения,Наличие патологоанатомических данных
count,299,276,300,240.000000,276.000000,242.000000,244,231,253,266,...,271.000000,267.000000,135,102.000000,299,300,300,300,300,300
unique,2,1,284,NaN,NaN,NaN,4,4,6,2,...,NaN,NaN,3,NaN,3,2,61,6,2,2
top,"Да, была операция",Взрослая лошадь,532349,NaN,NaN,NaN,Прохладные,нормальный,нормально-розовые,< 3 секунд,...,NaN,NaN,мутная,NaN,выжила,Да,0,0,0,Нет
freq,180,276,2,NaN,NaN,NaN,109,115,79,188,...,NaN,NaN,48,NaN,178,191,56,293,299,201
mean,NaN,NaN,NaN,38.167917,71.913043,30.417355,NaN,NaN,NaN,NaN,...,46.295203,24.456929,NaN,3.019608,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,0.732289,28.630557,17.642231,NaN,NaN,NaN,NaN,...,10.419335,27.475009,NaN,1.968567,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,35.400000,30.000000,8.000000,NaN,NaN,NaN,NaN,...,23.000000,3.300000,NaN,0.100000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,37.800000,48.000000,18.500000,NaN,NaN,NaN,NaN,...,38.000000,6.500000,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,38.200000,64.000000,24.500000,NaN,NaN,NaN,NaN,...,45.000000,7.500000,NaN,2.250000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,38.500000,88.000000,36.000000,NaN,NaN,NaN,NaN,...,52.000000,57.000000,NaN,3.900000,NaN,NaN,NaN,NaN,NaN,NaN


# Задание 2

1. Ректальная температура — выбросы отражают реальные патологические состояния. Оставить.  
2. Пульс — высокие значения соответствуют боли и шоку. Оставить.  
3. Частота дыхания — часть выбросов физиологична, часть шум. Ограничить до 80 при дальнейшей обработке.  
4. Гематокрит — повышенные значения отражают обезвоживание. Оставить.  
5. Общий белок — вероятны ошибки единиц измерения. Проверить и нормализовать при дальнейшей обработке.  
6. Белок при абдоминоцентезе — высокие значения отражают воспаление. Оставить.

In [209]:
# Задание 3
horses = prepare_data('horse_data.csv', attr, False)
horses = replace_values(horses, attr)
horses = clear_missing(horses, attr, False, 22)
horses = rename_columns(horses, attr)
display(horses)

,Операция,Возраст,Номер госпитализации,Ректальная температура,Пульс,Частота дыхания,Температура конечностей,Периферический пульс,Слизистые оболочки,Время капиллярного наполнения,...,Гематокрит,Общий белок,Внешний вид жидкости при абдоминоцентезе,Белок при абдоминоцентезе,Исход,Хирургическое поражение?,Тип поражения,Тип поражения,Тип поражения,Наличие патологоанатомических данных
0,"Нет, лечили без операции",Взрослая лошадь,530101,38.5,66.0,28.0,Прохладные,пониженный,Нет данных,≥ 3 секунд,...,45.0,8.4,Нет данных,2.8,умерла,Нет,11300,0,0,Нет
1,"Да, была операция",Взрослая лошадь,534817,39.2,88.0,20.0,Нет данных,Нет данных,бледно-цианотичные,< 3 секунд,...,50.0,85.0,мутная,2.0,эвтаназирована,Нет,2208,0,0,Нет
2,"Нет, лечили без операции",Взрослая лошадь,530334,38.3,40.0,24.0,Нормальные,нормальный,бледно-розовые,< 3 секунд,...,33.0,6.7,Нет данных,1.0,выжила,Нет,0,0,0,Да
3,"Да, была операция",Нет данных,5290409,39.1,164.0,84.0,Холодные,нормальный,тёмно-цианотичные,≥ 3 секунд,...,48.0,7.2,серозно-кровянистая,5.3,умерла,Да,2208,0,0,Да
4,"Нет, лечили без операции",Взрослая лошадь,530255,37.3,104.0,35.0,Нет данных,Нет данных,тёмно-цианотичные,≥ 3 секунд,...,74.0,7.4,Нет данных,2.8,умерла,Нет,4300,0,0,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"Да, была операция",Взрослая лошадь,533886,37.2,120.0,70.0,Холодные,Нет данных,бледно-цианотичные,≥ 3 секунд,...,55.0,65.0,Нет данных,2.0,эвтаназирована,Нет,3205,0,0,Нет
296,"Нет, лечили без операции",Взрослая лошадь,527702,37.2,72.0,24.0,Прохладные,повышенный,бледно-цианотичные,≥ 3 секунд,...,44.0,7.0,серозно-кровянистая,3.3,эвтаназирована,Да,2208,0,0,Да
297,"Да, была операция",Взрослая лошадь,529386,37.5,72.0,30.0,Холодные,пониженный,бледно-цианотичные,< 3 секунд,...,60.0,6.8,Нет данных,2.8,умерла,Да,3205,0,0,Нет
298,"Да, была операция",Взрослая лошадь,530612,36.5,100.0,24.0,Прохладные,пониженный,бледно-розовые,< 3 секунд,...,50.0,6.0,серозно-кровянистая,3.4,выжила,Да,2208,0,0,Да


# Задание 3

Я подозреваю, что данные пропущены не случайно, а из-за закономерностей (MAR). Но я ничего не понимаю в лошадиных коликах, и, по идее, удаление пропусков не приведёт к существенному искажению результатов. По крайней мере, если работать с 8 столбцами.

Для всех стобцов:
- Числовые заполнить средними по группе на основе Исхода. Исход предварительно очистить от пустых.
- Категориальные заменить "Нет данных"
